<a href="https://colab.research.google.com/github/pietroperrone01/UiO_AdvancedRemoteSensing/blob/main/Project1_sketch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Import packages
import ee
import geemap
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import altair as alt

In [5]:
# Run authentication
ee.Authenticate()
ee.Initialize(project='dulcet-iterator-470310-n0') # PUT YOUR API KEY (Project ID) HERE!

In [14]:
def mask_edge(image):
  edge = image.lt(-30.0)
  masked_image = image.mask().And(edge.Not())
  return image.updateMask(masked_image)

In [13]:
# define Svalbard area
svalbard_geom = ee.Geometry.Polygon(
    [[-2.434071, 81.099918],
     [46.394056, 81.099918],
     [46.394056, 74.304052],
     [-2.434071, 74.304052],
     [-2.434071, 81.099918]]
)

# Sentinel-1 collection for Svalbard
svalbard_img_hh = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterBounds(svalbard_geom)
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .select('HH')
    .map(mask_edge) # Reuse the mask_edge function from the previous cell
)

In [9]:
spring = ee.Filter.date('2015-03-01', '2015-04-20')
late_spring = ee.Filter.date('2015-04-21', '2015-06-10')
summer = ee.Filter.date('2015-06-11', '2015-08-31')

In [16]:
m = geemap.Map().set
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…